In [32]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


[nltk_data] Downloading package punkt to /Users/taolin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
text = ("""London (CNN)The UK's troubled response to the coronavirus pandemic became even more confused on Monday, as government guidance seemingly at odds with itself rolled out across England, pushing the four nations of the UK further apart.
From Monday, employers in England can ask staff to return to workplaces if they believe they are Covid-19 safe.
When the policy was announced by Prime Minister Boris Johnson last month, he was accused of "passing the buck on this big decision to employers" by the UK's Trades Union Congress (TUC). The TUC criticized the government for announcing such a move while the country's widely-criticized test and trace infrastructure was "still patchy."
Also launching on Monday is an eye-catching government scheme, "Eat Out to Help Out," aimed at getting restaurants back on their feet as the UK's furlough scheme comes to an end. Throughout August, people dining out from Monday to Wednesday are to be offered a 50% discount -- limited to £10 ($13) per person, and not including alcoholic drinks.
Both policies are part of a wider plan to get the UK's economy moving after months of lockdown kept many Brits stuck indoors and working from home while businesses in the hospitality industry that relied on their custom were forced to stop operating.
But these moves to soften coronavirus prevention measures, in order to restart the economy, come at the same time as cases are rising in Europe.
Last week, Johnson himself warned that "the risk is starting to bubble up again," on the continent, adding: "I'm afraid you are starting to see in some places the signs of a second wave of the pandemic."
In the UK several localized mini-lockdowns have been implemented, as fears of that second wave move from possible to probable.
These localized containment measures have created a particularly stark situation in areas like Manchester, where it is now against the rules to meet family members in a back garden, and yet absolutely fine, according to the rules, to go to a restaurant.
The move marks a significant shift towards prioritizing economic recovery ahead of other issues, including public health.
'I was too fat,' Boris Johnson says in UK launch to tackle obesity
This might worry some who heard Chris Whitty, England's chief medical officer, last week say that trade-offs over what can be opened have "probably reached near the limit" of what is possible.
Graham Medley, a member of the UK government's Scientific Advisory Group for Emergencies (SAGE), suggested over the weekend that one such trade-off could be pubs for schools.
After months of remote learning, many parents are desperate to send their youngsters back into the classroom next month. But while they might expect children's education to be prioritized ahead of people enjoying a few pints at the local bar, the current trend for favoring business suggests they may be disappointed.
A government spokesperson said on Monday that they expect all schools to be open from September. They stressed, however, that the UK's response would continue to be localized, where "you would assess the situation on the ground and take whatever steps were required to slow the spread of the virus."
On the specific question of choosing between pubs and schools, the spokesperson said, "we are committed to supporting the hospitality industry which has had a very tough time."
These measures apply only to England, as public health policy is a matter for the devolved governments of Scotland, Wales and Northern Ireland.
Throughout the pandemic, Johnson's government has been criticized by politicians and leaders in the three other nations for his perceived recklessness, most notably Scotland's First Minister, Nicola Sturgeon.
This perception of England mishandling the crisis has led to a surge for support in Scottish independence north of the border, though it is worth noting that Scotland's coronavirus record is not that much better than England's -- for every 100,000 people, 77 in Scotland have died with Covid-19 listed on their death certificate, versus 86 in England.
"""
)
print(text)

London (CNN)The UK's troubled response to the coronavirus pandemic became even more confused on Monday, as government guidance seemingly at odds with itself rolled out across England, pushing the four nations of the UK further apart.
From Monday, employers in England can ask staff to return to workplaces if they believe they are Covid-19 safe.
When the policy was announced by Prime Minister Boris Johnson last month, he was accused of "passing the buck on this big decision to employers" by the UK's Trades Union Congress (TUC). The TUC criticized the government for announcing such a move while the country's widely-criticized test and trace infrastructure was "still patchy."
Also launching on Monday is an eye-catching government scheme, "Eat Out to Help Out," aimed at getting restaurants back on their feet as the UK's furlough scheme comes to an end. Throughout August, people dining out from Monday to Wednesday are to be offered a 50% discount -- limited to £10 ($13) per person, and not i

In [13]:
sentences = sent_tokenize(text)

In [14]:
sentences

["London (CNN)The UK's troubled response to the coronavirus pandemic became even more confused on Monday, as government guidance seemingly at odds with itself rolled out across England, pushing the four nations of the UK further apart.",
 'From Monday, employers in England can ask staff to return to workplaces if they believe they are Covid-19 safe.',
 'When the policy was announced by Prime Minister Boris Johnson last month, he was accused of "passing the buck on this big decision to employers" by the UK\'s Trades Union Congress (TUC).',
 'The TUC criticized the government for announcing such a move while the country\'s widely-criticized test and trace infrastructure was "still patchy."',
 'Also launching on Monday is an eye-catching government scheme, "Eat Out to Help Out," aimed at getting restaurants back on their feet as the UK\'s furlough scheme comes to an end.',
 'Throughout August, people dining out from Monday to Wednesday are to be offered a 50% discount -- limited to £10 ($

In [17]:
# clean the formatting: lower case + no stopwords + no number
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

In [18]:
clean_sentences = [s.lower() for s in clean_sentences]

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taolin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [21]:
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [22]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences

['london cnn uk troubled response coronavirus pandemic became even confused monday government guidance seemingly odds rolled across england pushing four nations uk apart',
 'monday employers england ask staff return workplaces believe covid safe',
 'policy announced prime minister boris johnson last month accused passing buck big decision employers uk trades union congress tuc',
 'tuc criticized government announcing move country widely criticized test trace infrastructure still patchy',
 'also launching monday eye catching government scheme eat help aimed getting restaurants back feet uk furlough scheme comes end',
 'throughout august people dining monday wednesday offered discount limited per person including alcoholic drinks',
 'policies part wider plan get uk economy moving months lockdown kept many brits stuck indoors working home businesses hospitality industry relied custom forced stop operating',
 'moves soften coronavirus prevention measures order restart economy come time cas

In [23]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-08-03 15:54:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-03 15:54:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-03 15:54:53--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [25]:
 !unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [26]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [27]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [28]:
sentence_vectors

[array([-6.50227293e-02,  1.09421410e-01,  3.43319178e-01, -2.26886384e-02,
         1.38737723e-01,  5.41624874e-02, -1.34604290e-01,  3.98623534e-02,
        -1.09613970e-01, -3.88040133e-02,  5.49139120e-02, -1.45891145e-01,
         2.21378818e-01,  6.94088489e-02,  5.01423003e-03, -1.70713007e-01,
        -1.14701554e-01, -1.12785794e-01, -2.67588586e-01, -3.14723291e-02,
         3.60113859e-01,  9.45709646e-02,  2.76420861e-01, -9.81289893e-02,
         1.38596058e-01, -1.08159810e-01,  8.54343101e-02, -5.46665341e-02,
        -7.81204104e-02,  2.20463768e-01,  7.40597844e-02,  3.10095698e-01,
        -1.60909981e-01, -7.06497114e-03,  2.60706097e-02,  1.27191439e-01,
        -6.03058673e-02, -6.29825518e-03, -2.27996498e-01, -4.04012874e-02,
        -5.08621931e-01, -2.24788468e-02,  9.23385173e-02, -1.19434580e-01,
         2.26150453e-01, -2.39020437e-02,  2.44235575e-01,  6.99663907e-02,
         7.73831233e-02, -5.89912832e-01,  9.88816619e-02,  6.70485720e-02,
         9.5

In [29]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [33]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [34]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [36]:
scores #scores of the sentences

{0: 0.04684131833694706,
 1: 0.04517157784062214,
 2: 0.044519621671726205,
 3: 0.045655342920448136,
 4: 0.0465500224526511,
 5: 0.044171338577929764,
 6: 0.04656177630679394,
 7: 0.04505561717715347,
 8: 0.046083362588065536,
 9: 0.04344185808643158,
 10: 0.04693091310365716,
 11: 0.04499970381594276,
 12: 0.04664336757147845,
 13: 0.04601843998743189,
 14: 0.044501864116679515,
 15: 0.046370203258743414,
 16: 0.04528779267206317,
 17: 0.046021744223638285,
 18: 0.04575524329964504,
 19: 0.043176931076989604,
 20: 0.04474017459823863,
 21: 0.04550178631672317}

In [38]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [39]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

These localized containment measures have created a particularly stark situation in areas like Manchester, where it is now against the rules to meet family members in a back garden, and yet absolutely fine, according to the rules, to go to a restaurant.
London (CNN)The UK's troubled response to the coronavirus pandemic became even more confused on Monday, as government guidance seemingly at odds with itself rolled out across England, pushing the four nations of the UK further apart.
'I was too fat,' Boris Johnson says in UK launch to tackle obesity
This might worry some who heard Chris Whitty, England's chief medical officer, last week say that trade-offs over what can be opened have "probably reached near the limit" of what is possible.
Both policies are part of a wider plan to get the UK's economy moving after months of lockdown kept many Brits stuck indoors and working from home while businesses in the hospitality industry that relied on their custom were forced to stop operating.
A